In [2]:
# ---------- Path config: data is sibling of GRU folder ----------
from pathlib import Path

# "here" = the folder where this notebook/script lives (GRU/)
try:
    HERE = Path(__file__).resolve().parent
except NameError:
    # In Jupyter notebooks __file__ is not defined; use current working dir
    HERE = Path.cwd()

PROJECT_ROOT = HERE.parent         # one level up (where GRU and data are siblings)
DATA_DIR     = PROJECT_ROOT / "data"       # ../data
OUTPUT_DIR   = HERE / "output"             # keep outputs under GRU/output
PRICE_FILE   = DATA_DIR / "price_data_full.csv"

# (optional) sanity check prints
print("DATA_DIR  ->", DATA_DIR)
print("OUTPUT_DIR->", OUTPUT_DIR)
print("PRICE_FILE exists:", PRICE_FILE.exists())

DATA_DIR  -> f:\Master\Book\DSA5205\Project 2\code\qf_project2\data
OUTPUT_DIR-> f:\Master\Book\DSA5205\Project 2\code\qf_project2\GRU\output
PRICE_FILE exists: True


In [3]:
# ==============================================================
# Multi-Ticker | GRU (classification) → Online calibration → r̂
# Fixed split: Train/Valid/Test; train only on Train+Valid; rolling forecast on Test
# - Iterate tickers in CSV (multi-index columns: [PriceField, Ticker])
# - Features unified to the provided list (per-ticker, shifted by 1 to avoid leakage)
# - Keep online ridge calibration + online mean matching
# - "Scheme A" (rebasing-by-blocks) is for display/stats only
# - For each ticker:
#     * Train (≤ TRAIN_END) + Valid (TRAIN_END~VAL_END) → freeze weights
#     * Rolling forecast on Test (> VAL_END); write {TICKER}.csv -> [Date, PredictedPrice]
#     * Plot 1 chart "Actual vs Predicted (rebased)" and print OOS metrics (MSE, R2, IC, Hit, Sharpe)
# - Extra output: OOS_summary.csv (test metrics summary for all tickers)
# ==============================================================

import os, warnings
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch, torch.nn as nn
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm
from collections import deque

tqdm.monitor_interval = 0

# ---------------- Config ----------------
START_FROM_IDX = 0

PRICE_FILE = os.path.join(DATA_DIR, "price_data_full.csv")

START      = "2010-01-01"
END        = None  # to last complete day

# Fixed split boundaries
TRAIN_END = "2020-12-31"
VAL_END   = "2022-12-31"   # Testing is after VAL_END (exclusive)

# Sequence & minimum history
SEQ_LEN        = 20
MIN_TRAIN_DAYS = 40

# Training
BATCH_SIZE   = 512
LR           = 1e-3
WEIGHT_DECAY = 1e-4
MAX_EPOCHS   = 120
PATIENCE     = 12
HIDDEN_SIZE  = 64
NUM_LAYERS   = 2
DROPOUT      = 0.2
CLIP_NORM    = 1.0

# Calibration & safety caps
RIDGE_L2      = 5e-6
B_CAP         = 0.17
RHAT_CAP      = 0.023
CAL_WIN       = 40
MIN_CAL_SAMPLES = 40

# Online mean matching
MEAN_MATCH_WIN = 40
MEAN_MATCH_MIN = 20

# Display-only: Scheme A (rebasing by blocks)
REBASE_DAYS_FOR_CURVE = 20

np.random.seed(42)
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
if device.type == "cuda":
    try: torch.set_float32_matmul_precision("high")
    except: pass
    torch.backends.cudnn.benchmark = True

def last_complete_day():
    return (pd.Timestamp.today().normalize() - pd.Timedelta(days=1)).strftime("%Y-%m-%d")
if END is None: END = last_complete_day()

# ---------------- Simple AdamW (pure PyTorch, decoupled weight decay) ----------------
class SimpleAdamW:
    def __init__(self, params, lr=1e-3, weight_decay=1e-4, betas=(0.9, 0.999), eps=1e-8):
        plist = list(params)
        self.params = [p for p in plist if isinstance(p, torch.Tensor) and p.requires_grad]
        if not self.params:
            raise ValueError("Model has no trainable parameters.")
        self.lr, self.wd, self.betas, self.eps = lr, weight_decay, betas, eps
        self.state = {}

    def zero_grad(self, set_to_none=True):
        for p in self.params:
            if p.grad is not None:
                if set_to_none: p.grad = None
                else: p.grad.detach_(); p.grad.zero_()

    @torch.no_grad()
    def step(self):
        b1, b2 = self.betas
        for p in self.params:
            g = p.grad
            if g is None: continue
            if self.wd:
                p.data.mul_(1 - self.lr * self.wd)
            st = self.state.get(p)
            if st is None:
                st = self.state[p] = {
                    "t": 0,
                    "m": torch.zeros_like(p, memory_format=torch.preserve_format),
                    "v": torch.zeros_like(p, memory_format=torch.preserve_format),
                }
            st["t"] += 1; t = st["t"]
            st["m"].mul_(b1).add_(g, alpha=1 - b1)
            st["v"].mul_(b2).addcmul_(g, g, value=1 - b2)
            m_hat = st["m"] / (1 - b1**t)
            v_hat = st["v"] / (1 - b2**t)
            p.data.addcdiv_(m_hat, v_hat.sqrt().add_(self.eps), value=-self.lr)

# ---------------- Indicators & helpers ----------------
def rsi(series, n=14):
    d = series.diff()
    up = d.clip(lower=0); down = -d.clip(upper=0)
    ma_up = up.rolling(n, min_periods=n).mean()
    ma_dn = down.rolling(n, min_periods=n).mean()
    rs = ma_up / ma_dn
    out = 100 - (100/(1+rs))
    return out/100.0  # scaled to 0~1

def macd(close, fast=12, slow=26, signal=9):
    ema_f = close.ewm(span=fast, adjust=False).mean()
    ema_s = close.ewm(span=slow, adjust=False).mean()
    line  = ema_f - ema_s
    sig   = line.ewm(span=signal, adjust=False).mean()
    hist  = line - sig
    return line, sig, hist

def average_true_range(high, low, close, n=14):
    prev_close = close.shift(1)
    tr = pd.concat([
        (high - low).abs(),
        (high - prev_close).abs(),
        (low  - prev_close).abs()
    ], axis=1).max(axis=1)
    atr = tr.rolling(n, min_periods=n).mean()
    return atr

def online_mean_match(curr_rhat, ret_hist: deque, rhat_hist: deque):
    if MEAN_MATCH_WIN is None or len(ret_hist) < MEAN_MATCH_MIN or len(rhat_hist) == 0:
        return float(curr_rhat)
    mu_true = float(np.mean(ret_hist))
    mu_pred = float(np.mean(rhat_hist))
    return float(curr_rhat + (mu_true - mu_pred))

# ---------------- GRU ----------------
class GRUTrend(nn.Module):
    def __init__(self, in_dim, hidden=64, layers=2, dropout=0.2):
        super().__init__()
        self.gru = nn.GRU(input_size=in_dim, hidden_size=hidden, num_layers=layers,
                          dropout=(dropout if layers>1 else 0.0), batch_first=True)
        self.head = nn.Sequential(nn.Dropout(dropout), nn.Linear(hidden, 1))
    def forward(self, x):
        y, _ = self.gru(x)        # (B,T,H)
        last = y[:, -1, :]
        logit = self.head(last)   # (B,1)
        return logit.squeeze(1)

def make_seq_2d_to_3d(X2d: np.ndarray, seq_len: int):
    Xs = []
    for i in range(seq_len-1, len(X2d)):
        Xs.append(X2d[i-seq_len+1:i+1, :])
    return np.stack(Xs) if len(Xs)>0 else np.zeros((0, seq_len, X2d.shape[1]), dtype=X2d.dtype)

def align_y(y: np.ndarray, seq_len: int):
    return y[seq_len-1:]

def get_by_key(series: pd.Series, key):
    return series.iloc[key] if isinstance(key, (int, np.integer)) else series.loc[key]

# ---------------- Online ridge calibrator (robust, sliding window) ----------------
class OnlineCalibrator:
    """
    Keeps a sliding window of (z, r), returns closed-form ridge solution for (a_t, b_t).
    z = p - 0.5 ; r is next-day realized return (revealed with 1-day lag).
    """
    def __init__(self, ridge=1e-6, cap_b=0.10, win=252):
        self.ridge = ridge
        self.cap_b = cap_b
        self.win   = win if (win is None or win > 0) else None

        # Do not use maxlen to avoid implicit pops that desync running sums.
        self.buf = deque()

        # Running sums
        self.n = 0
        self.sz = 0.0; self.sr = 0.0; self.szz = 0.0; self.szr = 0.0

    def _push(self, z, r):
        self.buf.append((z, r))
        self.n  += 1
        self.sz += z
        self.sr += r
        self.szz += z*z
        self.szr += z*r

    def _pop_left(self):
        if self.win is None:
            return
        while self.n > self.win:
            oldz, oldr = self.buf.popleft()
            self.n  -= 1
            self.sz -= oldz
            self.sr -= oldr
            self.szz -= oldz*oldz
            self.szr -= oldz*oldr

    def add(self, z, r):
        self._push(float(z), float(r))
        self._pop_left()

    def fit_from_arrays(self, z_arr, r_arr):
        if len(z_arr) != len(r_arr):
            raise ValueError("z_arr and r_arr must have the same length")
        if self.win is not None and len(z_arr) > self.win:
            z_arr = z_arr[-self.win:]
            r_arr = r_arr[-self.win:]

        # Rebuild buffer and recompute sums
        self.buf.clear()
        self.buf.extend((float(z), float(r)) for z, r in zip(z_arr, r_arr))
        self.n = len(self.buf)

        if self.n == 0:
            self.sz = self.sr = self.szz = self.szr = 0.0
            return

        zs = [z for z, _ in self.buf]
        rs = [r for _, r in self.buf]
        self.sz  = float(np.sum(zs))
        self.sr  = float(np.sum(rs))
        self.szz = float(np.dot(zs, zs))
        self.szr = float(np.dot(zs, rs))

    def coef(self):
        if self.n < MIN_CAL_SAMPLES:
            return None
        n, sz, sr, szz, szr = self.n, self.sz, self.sr, self.szz, self.szr
        a00 = n + self.ridge
        a01 = sz
        a11 = szz + self.ridge
        b0, b1 = sr, szr
        det = a00*a11 - a01*a01
        if det <= 1e-18:
            return 0.0, 0.0
        a = ( b0*a11 - b1*a01) / det
        b = ( a00*b1 - a01*b0) / det
        b = float(np.clip(b, -self.cap_b, self.cap_b))
        return float(a), b

# ---------------- Training with validation (early stopping) ----------------
def train_with_val(X_train3, y_train1, X_val3, y_val1, n_feat_for_model):
    ds_tr = torch.utils.data.TensorDataset(
        torch.from_numpy(X_train3.astype(np.float32)),
        torch.from_numpy(y_train1.astype(np.float32)))
    ds_va = torch.utils.data.TensorDataset(
        torch.from_numpy(X_val3.astype(np.float32)),
        torch.from_numpy(y_val1.astype(np.float32)))
    pin = (device.type == "cuda")
    dl_tr = torch.utils.data.DataLoader(ds_tr, batch_size=BATCH_SIZE, shuffle=True,
                                        drop_last=False, pin_memory=pin)
    dl_va = torch.utils.data.DataLoader(ds_va, batch_size=2048, shuffle=False,
                                        drop_last=False, pin_memory=pin)

    model = GRUTrend(n_feat_for_model, hidden=HIDDEN_SIZE, layers=NUM_LAYERS, dropout=DROPOUT).to(device)
    pos_w = max(1.0, (len(y_train1)-y_train1.sum())/max(1.0, y_train1.sum()))
    loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_w], device=device))
    opt = SimpleAdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

    best_state, best_val, bad = None, float("inf"), 0
    for _ in range(MAX_EPOCHS):
        model.train()
        for xb, yb in dl_tr:
            xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
            opt.zero_grad(set_to_none=True)
            loss = loss_fn(model(xb), yb)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), CLIP_NORM)
            opt.step()

        model.eval(); vs=[]
        with torch.no_grad():
            for xb, yb in dl_va:
                xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
                vs.append(loss_fn(model(xb), yb).item())
        v = float(np.mean(vs))
        if v < best_val - 1e-9:
            best_val, bad = v, 0
            best_state = {k: w.detach().clone() for k, w in model.state_dict().items()}
        else:
            bad += 1
            if bad >= PATIENCE: break

    if best_state is not None: model.load_state_dict(best_state)
    model.eval()
    return model

@torch.no_grad()
def predict_proba(model, X3):
    if len(X3) == 0: return np.array([], dtype=np.float32)
    tens = torch.from_numpy(X3.astype(np.float32)).to(device)
    return 1/(1+np.exp(-model(tens).cpu().numpy()))

# ---------------- Feature builder (unified list) ----------------
def build_unified_features(ticker, close_s, open_s, high_s, low_s, vol_s):
    """
    Returns a DataFrame with columns named exactly as:
      f'{ticker}_ret_5d', f'{ticker}_ret_20d', f'{ticker}_ret_60d', f'{ticker}_mom_accel',
      f'{ticker}_rsi', f'{ticker}_macd_hist', f'{ticker}_ma20_bias', f'{ticker}_bb_position', f'{ticker}_ma_alignment',
      f'{ticker}_vol_20d', f'{ticker}_vol_60d', f'{ticker}_atr',
      f'{ticker}_volume_ratio', f'{ticker}_volume_mom',
      f'{ticker}_52w_position', f'{ticker}_from_52w_high',
      f'{ticker}_hl_spread'
    All features are shifted by 1 day to avoid look-ahead bias.
    """
    eps = 1e-12

    # Returns for momentum & volatility
    ret_1 = close_s.pct_change(1)
    ret_5  = close_s.pct_change(5)
    ret_20 = close_s.pct_change(20)
    ret_60 = close_s.pct_change(60)

    # Momentum group
    mom_accel = ret_5 - ret_20  # short-term momentum minus medium-term

    # Technical indicators
    rsi14 = rsi(close_s, 14)  # scaled 0~1
    _, _, macd_hist = macd(close_s)  # 12-26-9
    ma20 = close_s.rolling(20).mean()
    ma20_bias = close_s / (ma20 + eps) - 1.0

    std20 = close_s.rolling(20).std()
    upper = ma20 + 2*std20
    lower = ma20 - 2*std20
    bb_denom = (upper - lower).replace(0, np.nan)
    bb_pos = (close_s - lower) / bb_denom
    bb_pos = bb_pos.clip(0.0, 1.0).fillna(0.5)

    ma5  = close_s.rolling(5).mean()
    ma10 = close_s.rolling(10).mean()
    ma_align = 0.5*(ma5/(ma10+eps) - 1.0) + 0.5*(ma10/(ma20+eps) - 1.0)

    # Volatility
    vol_20d = ret_1.rolling(20).std()
    vol_60d = ret_1.rolling(60).std()
    atr14 = average_true_range(high_s, low_s, close_s, 14) / (close_s + eps)

    # Volume
    v_ma20 = vol_s.rolling(20).mean()
    volume_ratio = vol_s / (v_ma20 + eps)
    volume_mom   = (vol_s.rolling(5).mean() / (v_ma20 + eps)) - 1.0

    # Price position
    hi_52w = close_s.rolling(252).max()
    lo_52w = close_s.rolling(252).min()
    range_52w = (hi_52w - lo_52w).replace(0, np.nan)
    pos_52w = (close_s - lo_52w) / range_52w
    pos_52w = pos_52w.clip(0.0, 1.0).fillna(0.5)
    from_52w_high = close_s / (hi_52w + eps) - 1.0

    # Microstructure
    hl_spread = np.log(high_s/low_s)

    # Assemble
    cols = {
        f"{ticker}_ret_5d":        ret_5,
        f"{ticker}_ret_20d":       ret_20,
        f"{ticker}_ret_60d":       ret_60,
        f"{ticker}_mom_accel":     mom_accel,

        f"{ticker}_rsi":           rsi14,
        f"{ticker}_macd_hist":     macd_hist,
        f"{ticker}_ma20_bias":     ma20_bias,
        f"{ticker}_bb_position":   bb_pos,
        f"{ticker}_ma_alignment":  ma_align,

        f"{ticker}_vol_20d":       vol_20d,
        f"{ticker}_vol_60d":       vol_60d,
        f"{ticker}_atr":           atr14,

        f"{ticker}_volume_ratio":  volume_ratio,
        f"{ticker}_volume_mom":    volume_mom,

        f"{ticker}_52w_position":  pos_52w,
        f"{ticker}_from_52w_high": from_52w_high,

        f"{ticker}_hl_spread":     hl_spread,
    }
    feat = pd.DataFrame(cols)
    feat = feat.replace([np.inf, -np.inf], np.nan)
    feat = feat.shift(1)  # critical: avoid look-ahead
    return feat

# ---------------- Load all data ----------------
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
if not os.path.exists(PRICE_FILE):
    raise FileNotFoundError(f"File not found: {PRICE_FILE}")

df = pd.read_csv(PRICE_FILE, header=[0,1], index_col=0)
if isinstance(df.index[0], str) and str(df.index[0]).strip().lower() == "date":
    df = df.iloc[1:]
df.index = pd.to_datetime(df.index, errors="coerce")
df = df[~df.index.isna()].sort_index()
df = df.loc[(df.index >= START) & (df.index <= END)]

all_tickers = df.columns.get_level_values(1).unique().tolist()
print(f"Tickers detected: {len(all_tickers)}; start from #{START_FROM_IDX+1}")

summary = []

# ---------------- Main loop per ticker ----------------
for TICKER in tqdm(all_tickers[START_FROM_IDX:], desc=f"All tickers [{START_FROM_IDX+1}→{len(all_tickers)}]"):
    try:
        close_s = pd.to_numeric(df[("Close",  TICKER)], errors="coerce")
        open_s  = pd.to_numeric(df[("Open",   TICKER)], errors="coerce")
        high_s  = pd.to_numeric(df[("High",   TICKER)], errors="coerce")
        low_s   = pd.to_numeric(df[("Low",    TICKER)], errors="coerce")
        vol_s   = pd.to_numeric(df[("Volume", TICKER)], errors="coerce")
    except KeyError:
        print(f"\n[{TICKER}] missing fields, skip."); 
        continue

    # Minimum length check
    if close_s.dropna().shape[0] < (SEQ_LEN + MIN_TRAIN_DAYS + 50):
        print(f"\n[{TICKER}] too few points, skip."); 
        continue

    # Targets
    y_ret = close_s.shift(-1)/close_s - 1.0
    y_cls = (y_ret > 0).astype(int)

    # Unified features (all shifted by 1)
    feat = build_unified_features(TICKER, close_s, open_s, high_s, low_s, vol_s)

    # Align and drop NaN
    data = pd.concat([feat, y_ret.rename("y_ret"), y_cls.rename("y_cls"), close_s.rename("close")], axis=1).dropna()
    if data.shape[0] < (SEQ_LEN + MIN_TRAIN_DAYS + 10):
        print(f"\n[{TICKER}] not enough aligned samples, skip."); 
        continue

    X_all   = data.drop(columns=['y_ret','y_cls','close'])
    ret_all = data['y_ret']
    cls_all = data['y_cls'].astype(int)
    close_all = data['close']
    dates = X_all.index
    n_feat = X_all.shape[1]

    # Date-based split
    train_idx = dates[dates <= pd.Timestamp(TRAIN_END)]
    val_idx   = dates[(dates > pd.Timestamp(TRAIN_END)) & (dates <= pd.Timestamp(VAL_END))]
    test_idx  = dates[dates > pd.Timestamp(VAL_END)]

    if len(train_idx) < SEQ_LEN + MIN_TRAIN_DAYS or len(val_idx) < max(5, SEQ_LEN//2) or len(test_idx) < SEQ_LEN:
        print(f"\n[{TICKER}] split too short (train/val/test), skip."); 
        continue

    X_train = X_all.loc[train_idx]; y_train_cls = cls_all.loc[train_idx]; y_train_ret = ret_all.loc[train_idx]
    X_val   = X_all.loc[val_idx];   y_val_cls   = cls_all.loc[val_idx];   y_val_ret   = ret_all.loc[val_idx]
    X_test  = X_all.loc[test_idx];  y_test_ret  = ret_all.loc[test_idx]   # used for OOS metrics

    # Scaler fit only on training
    scaler = StandardScaler().fit(X_train.values)
    Xtr2 = scaler.transform(X_train.values); Xva2 = scaler.transform(X_val.values)

    # Sequences
    Xtr3 = make_seq_2d_to_3d(Xtr2, SEQ_LEN); Xva3 = make_seq_2d_to_3d(Xva2, SEQ_LEN)
    ytr1 = align_y(y_train_cls.values.astype(np.float32), SEQ_LEN)
    yva1 = align_y(y_val_cls.values.astype(np.float32),   SEQ_LEN)

    # Train + early stop on validation
    model = train_with_val(Xtr3, ytr1, Xva3, yva1, n_feat_for_model=n_feat)

    # Seed online calibrator using train+valid z,r (model frozen)
    z_seed, r_seed = [], []
    p_tr = predict_proba(model, make_seq_2d_to_3d(Xtr2, SEQ_LEN))
    r_tr = align_y(y_train_ret.values.astype(np.float32), SEQ_LEN)
    z_seed.append(p_tr - 0.5); r_seed.append(r_tr)

    p_va = predict_proba(model, make_seq_2d_to_3d(Xva2, SEQ_LEN))
    r_va = align_y(y_val_ret.values.astype(np.float32), SEQ_LEN)
    z_seed.append(p_va - 0.5); r_seed.append(r_va)

    z_seed = np.concatenate(z_seed) if len(z_seed)>0 else np.zeros(0)
    r_seed = np.concatenate(r_seed) if len(r_seed)>0 else np.zeros(0)

    cal = OnlineCalibrator(ridge=RIDGE_L2, cap_b=B_CAP, win=CAL_WIN)
    if len(z_seed) > 0:
        cal.fit_from_arrays(z_seed, r_seed)

    # Rolling forecast on Test (model params frozen)
    pred_prob = pd.Series(index=test_idx, dtype=float)
    pred_rhat = pd.Series(index=test_idx, dtype=float)

    ret_hist  = deque(maxlen=MEAN_MATCH_WIN)
    rhat_hist = deque(maxlen=MEAN_MATCH_WIN)

    z_prev, prev_j = None, None
    for j in test_idx:
        X_hist = X_all.loc[:j].values
        if len(X_hist) < SEQ_LEN: 
            continue
        X_hist_sc = scaler.transform(X_hist)
        X_last3   = make_seq_2d_to_3d(X_hist_sc, SEQ_LEN)[-1:,:,:]

        p = float(predict_proba(model, X_last3)[0])
        z = p - 0.5

        coef = cal.coef()
        a_t, b_t = (0.0, 0.0) if coef is None else coef

        rhat = a_t + b_t * z
        rhat = online_mean_match(rhat, ret_hist, rhat_hist)
        rhat = float(np.clip(rhat, -RHAT_CAP, RHAT_CAP))

        pred_prob.loc[j] = p
        pred_rhat.loc[j] = rhat

        # Update calibrator and mean-match buffers with yesterday's realized data
        if z_prev is not None and prev_j is not None:
            cal.add(z_prev, float(get_by_key(ret_all, prev_j)))
            ret_hist.append(float(get_by_key(ret_all, prev_j)))
            rhat_hist.append(float(pred_rhat.loc[prev_j]))
        z_prev, prev_j = z, j

    # Build synthetic price on Test (shift(1) to apply on next day)
    valid = pred_prob.dropna().index
    if len(valid)==0:
        print(f"\n[{TICKER}] no valid test predictions, skip.")
        continue

    ret_oos   = y_test_ret.loc[valid]
    close_oos = close_all.loc[valid]

    P0 = float(close_oos.iloc[0])
    pred_factor = (1.0 + pred_rhat.loc[valid]).cumprod().shift(1).fillna(1.0)
    pred_price  = pd.Series(P0, index=pred_factor.index) * pred_factor

    # Scheme A rebasing (display only)
    dfp = pd.DataFrame({"actual": close_oos, "pred": pred_price}).dropna().copy()
    blocks_curve = np.arange(len(dfp)) // REBASE_DAYS_FOR_CURVE
    def _rebase_block(g: pd.DataFrame):
        s = float(g["actual"].iloc[0] / g["pred"].iloc[0])
        g["pred_rb"] = g["pred"] * s
        return g
    dfp = dfp.groupby(blocks_curve, group_keys=False).apply(_rebase_block)
    pred_price_rb = dfp["pred_rb"]

    # Save test csv: Date, PredictedPrice
    out = pd.DataFrame({"Date": valid, "PredictedPrice": pred_price.loc[valid].values})
    out.to_csv(os.path.join(OUTPUT_DIR, f"{TICKER}.csv"), index=False)
    print(f"\n[{TICKER}] saved -> {TICKER}.csv  (test period)")

    # For metrics
    price_actual = close_oos
    price_pred   = pred_price.loc[valid]

    # OOS metrics on returns
    price_err = price_actual - price_pred
    mse_model = float((price_err ** 2).mean())
    mse_base  = float(((price_actual - price_actual.mean()) ** 2).mean())
    r2_oos    = 1 - mse_model / mse_base if mse_base > 0 else np.nan
    ic        = float(pd.Series(price_pred).corr(price_actual))

    price_diff = price_actual.diff()
    pred_diff  = price_pred.diff()
    common_idx = price_diff.dropna().index.intersection(pred_diff.dropna().index)
    hit        = float((np.sign(pred_diff.loc[common_idx]) == np.sign(price_diff.loc[common_idx])).mean())

    # mse_model = float(((ret_oos - pred_rhat.loc[valid])**2).mean())
    # mse_base  = float(((ret_oos - ret_oos.mean())**2).mean())
    # r2_oos    = 1 - mse_model/mse_base if mse_base>0 else np.nan
    # ic        = float(pd.Series(pred_rhat.loc[valid]).corr(ret_oos))
    # hit       = float((np.sign(pred_rhat.loc[valid])==np.sign(ret_oos)).mean())

    # Directional Sharpe (reference only)
    str_ret = np.sign(pred_rhat.loc[valid]) * ret_oos
    ann_ret = (1+str_ret.mean())**252 - 1
    ann_vol = str_ret.std(ddof=0) * np.sqrt(252)
    sharpe  = float(ann_ret/ann_vol) if ann_vol>0 else np.nan

    print(f"[{TICKER}] TEST  MSE={mse_model:.3e} | R2={r2_oos: .3f} | IC={ic: .3f} | Hit={hit: .3f} | Sharpe={sharpe: .2f}")

    summary.append([TICKER, mse_model, r2_oos, ic, hit, sharpe, valid[0].date(), valid[-1].date(), len(valid)])

    # Plot: Actual vs Predicted (rebased)
    plt.figure(figsize=(10,3.2))
    plt.plot(close_oos.index, close_oos.values, label="Actual Close (USD)")
    plt.plot(pred_price_rb.index, pred_price_rb.values,
             label=f"Predicted (rebased {REBASE_DAYS_FOR_CURVE}d)")
    plt.title(f"{TICKER} | Actual vs Predicted (TEST)")
    plt.ylabel("USD"); plt.legend(); plt.grid(True, alpha=0.3)
    fn = os.path.join(OUTPUT_DIR, f"{TICKER}_TEST_curve.png")
    plt.tight_layout(); plt.savefig(fn, dpi=140); plt.close()
    print(f"[{TICKER}] saved -> {os.path.basename(fn)}")

    if device.type == "cuda":
        torch.cuda.empty_cache()

# ---------------- Summary ----------------
sum_df = pd.DataFrame(summary, columns=[
    "Ticker","MSE","R2_TEST","IC","HitRate","Sharpe","TEST_start","TEST_end","#TEST_days"
])
sum_df = sum_df.sort_values("R2_TEST", ascending=False)
sum_df.to_csv(os.path.join(OUTPUT_DIR, "OOS_summary.csv"), index=False)
print("\nSaved OOS_summary.csv")
print(sum_df.head(20).to_string(index=False))


Device: cuda
Tickers detected: 12; start from #1


All tickers [1→12]:   0%|          | 0/12 [00:00<?, ?it/s]


[AAPL] saved -> AAPL.csv  (test period)
[AAPL] TEST  MSE=2.985e+03 | R2=-2.296 | IC= 0.910 | Hit= 0.494 | Sharpe=-0.29
[AAPL] saved -> AAPL_TEST_curve.png

[AMZN] saved -> AMZN.csv  (test period)
[AMZN] TEST  MSE=8.707e+02 | R2= 0.515 | IC= 0.949 | Hit= 0.494 | Sharpe=-0.35
[AMZN] saved -> AMZN_TEST_curve.png

[AVGO] saved -> AVGO.csv  (test period)
[AVGO] TEST  MSE=1.343e+03 | R2= 0.788 | IC= 0.966 | Hit= 0.494 | Sharpe= 0.69
[AVGO] saved -> AVGO_TEST_curve.png

[GOOGL] saved -> GOOGL.csv  (test period)
[GOOGL] TEST  MSE=1.630e+02 | R2= 0.875 | IC= 0.938 | Hit= 0.496 | Sharpe= 0.12
[GOOGL] saved -> GOOGL_TEST_curve.png

[META] saved -> META.csv  (test period)
[META] TEST  MSE=4.153e+03 | R2= 0.872 | IC= 0.968 | Hit= 0.511 | Sharpe= 0.73
[META] saved -> META_TEST_curve.png

[MSFT] saved -> MSFT.csv  (test period)
[MSFT] TEST  MSE=4.199e+02 | R2= 0.922 | IC= 0.967 | Hit= 0.496 | Sharpe= 0.02
[MSFT] saved -> MSFT_TEST_curve.png

[NFLX] saved -> NFLX.csv  (test period)
[NFLX] TEST  MSE=8

In [3]:
# ===== plots_all_curves_price_metrics.py =====
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------------------- Config (edit paths if needed) ----------------------
PRICE_FILE  = os.path.join(DATA_DIR, "price_data_full.csv")
OUT_DIR     = os.path.join(OUTPUT_DIR, "diag_price_all")
WIN         = 60  # rolling window for metrics
MAX_TICKERS = 5

os.makedirs(OUT_DIR, exist_ok=True)

# ---------------------- Data loading helpers ----------------------
def read_price_panel(price_file: str) -> pd.DataFrame:
    """
    Read panel CSV with a two-level header [PriceField, Ticker].
    Returns a DataFrame indexed by datetime, columns are MultiIndex.
    """
    df = pd.read_csv(price_file, header=[0, 1], index_col=0)
    if isinstance(df.index[0], str) and str(df.index[0]).strip().lower() == "date":
        df = df.iloc[1:]
    df.index = pd.to_datetime(df.index, errors="coerce")
    df = df[~df.index.isna()].sort_index()
    return df

def load_actual_close(df_panel: pd.DataFrame, ticker: str) -> pd.Series:
    """Get Close series for a ticker (float Series indexed by date)."""
    return pd.to_numeric(df_panel[("Close", ticker)], errors="coerce")

def load_predicted_price(output_dir: str, ticker: str) -> pd.Series:
    """
    Read {OUTPUT_DIR}/{TICKER}.csv with columns [Date, PredictedPrice].
    Return a float Series indexed by Date.
    """
    fp = os.path.join(output_dir, f"{ticker}.csv")
    if not os.path.exists(fp):
        raise FileNotFoundError(fp)
    g = pd.read_csv(fp)
    if "Date" not in g.columns or "PredictedPrice" not in g.columns:
        raise ValueError(f"{fp} must contain columns: Date, PredictedPrice")
    g["Date"] = pd.to_datetime(g["Date"])
    g = g.dropna(subset=["Date"]).sort_values("Date")
    return pd.Series(g["PredictedPrice"].astype(float).values, index=g["Date"], name="PredictedPrice")

# ---------------------- Plotting helpers ----------------------
def make_color_cycle(n: int):
    """
    Build an aesthetic color list long enough for n tickers:
    use tab20, tab20b, tab20c first, then fallback to HSV.
    """
    palettes = []
    for name in ["tab20", "tab20b", "tab20c"]:
        cmap = plt.get_cmap(name)
        palettes.extend(list(cmap.colors))
    if n <= len(palettes):
        return palettes[:n]
    # fallback: evenly spaced HSV
    return [plt.cm.hsv(i / max(1, n)) for i in range(n)]

def plot_multicurve(series_map, title, ylabel, fname, use_logy=False, legend_cols=4):
    """
    Plot multiple time series (dict: {ticker: pd.Series}) on one figure.
    """
    plt.figure(figsize=(12, 6))
    colors = make_color_cycle(len(series_map))
    for (ticker, s), c in zip(series_map.items(), colors):
        s = s.dropna()
        if len(s) == 0:
            continue
        plt.plot(s.index, s.values, label=ticker, lw=1.6, alpha=0.95, color=c)
    if use_logy:
        plt.yscale("log")
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel("Date")
    plt.grid(True, alpha=0.3)
    # place legend at bottom to reduce occlusion
    plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.12), ncol=legend_cols, fontsize=8)
    plt.tight_layout()
    fp = os.path.join(OUT_DIR, fname)
    plt.savefig(fp, dpi=160, bbox_inches="tight")
    plt.close()
    print(f"Saved: {fp}")

# ---------------------- Core computation ----------------------
def compute_rolling_metrics_all(df_panel: pd.DataFrame, tickers: list, win: int = 60):
    """
    For each ticker, compute:
      - hit_rolling (price-diff direction hit-rate, rolling mean)
      - ic_rolling  (price-level correlation, rolling)
      - mse_rolling (price-level squared error rolling mean)
      - rel_err     (Actual/Pred - 1), not rolling (raw curve)
    Returns four dicts mapping ticker -> pd.Series.
    """
    hit_curves = {}
    ic_curves  = {}
    mse_curves = {}
    rel_curves = {}

    for tkr in tickers:
        # Load price series
        try:
            pa = load_actual_close(df_panel, tkr).dropna()
        except KeyError:
            print(f"[{tkr}] missing Close in PRICE_FILE, skipped.")
            continue
        try:
            pp = load_predicted_price(OUTPUT_DIR, tkr).dropna()
        except (FileNotFoundError, ValueError):
            print(f"[{tkr}] missing predicted CSV, skipped.")
            continue

        # Align by common dates
        idx = pa.index.intersection(pp.index)
        if len(idx) < max(40, win + 5):
            print(f"[{tkr}] too few overlapping dates, skipped.")
            continue
        pa = pa.loc[idx].astype(float)
        pp = pp.loc[idx].astype(float)

        # Rolling MSE on price
        price_res = (pa - pp)
        mse_s = (price_res ** 2).rolling(win).mean()

        # Rolling IC on price (correlation)
        ic_s = pp.rolling(win).corr(pa)

        # Rolling Hit on price-diff direction
        pa_diff = pa.diff()
        pp_diff = pp.diff()
        diff_idx = pa_diff.dropna().index.intersection(pp_diff.dropna().index)
        hit_series = (np.sign(pp_diff.loc[diff_idx]) == np.sign(pa_diff.loc[diff_idx])).astype(float)
        hit_s = hit_series.rolling(win).mean()

        # Relative price error (Actual/Pred - 1), raw series
        rel_err = (pa / pp) - 1.0

        hit_curves[tkr] = hit_s
        ic_curves[tkr]  = ic_s
        mse_curves[tkr] = mse_s
        rel_curves[tkr] = rel_err

    return hit_curves, ic_curves, mse_curves, rel_curves

# ---------------------- Main ----------------------
if __name__ == "__main__":
    # Load panel and discover tickers with prediction CSVs
    panel = read_price_panel(PRICE_FILE)

    # tickers listed by predicted CSVs (ensures availability)
    out_csvs = [f for f in os.listdir(OUTPUT_DIR) if f.endswith(".csv") and f != "OOS_summary.csv"]
    tickers = sorted({os.path.splitext(f)[0] for f in out_csvs})

    if len(tickers) == 0:
        raise RuntimeError("No {TICKER}.csv found in OUTPUT_DIR.")

    # Only use first MAX_TICKERS for plotting
    selected = tickers[:MAX_TICKERS]
    print(f"Tickers discovered: {len(tickers)}  → using first {len(selected)}: {selected}")

    # Only compute metrics for selected tickers
    hit_map, ic_map, mse_map, rel_map = compute_rolling_metrics_all(panel, selected, win=WIN)

    # Plot (file names can have a prefix for distinction)
    plot_multicurve(hit_map, f"Rolling Hit-rate on Price Diff ({WIN}d)", f"Hit({WIN}d)", "TOP5_hit_60d.png")
    plot_multicurve(ic_map,  f"Rolling IC on Price Level ({WIN}d)",    f"IC({WIN}d)",  "TOP5_ic_60d.png")
    plot_multicurve(mse_map, f"Rolling MSE on Price ({WIN}d)",         f"MSE({WIN}d)", "TOP5_mse_60d.png", use_logy=False)
    plot_multicurve(rel_map, "Relative Price Error (Actual/Pred − 1)", "Rel. error",   "TOP5_relative_price_error.png")


Tickers discovered: 12  → using first 5: ['AAPL', 'AMZN', 'AVGO', 'GOOGL', 'META']
Saved: f:\Master\Book\DSA5205\Project 2\code\qf_project2\GRU\output\diag_price_all\TOP5_hit_60d.png
Saved: f:\Master\Book\DSA5205\Project 2\code\qf_project2\GRU\output\diag_price_all\TOP5_ic_60d.png
Saved: f:\Master\Book\DSA5205\Project 2\code\qf_project2\GRU\output\diag_price_all\TOP5_mse_60d.png
Saved: f:\Master\Book\DSA5205\Project 2\code\qf_project2\GRU\output\diag_price_all\TOP5_relative_price_error.png
